In [ ]:
# Downloading dependencies 
!pip install pandas seaborn matplotlib

In [2]:
# Importing dependencies
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 

In [3]:
# Reading datasets for analysis
df_Patient = pd.read_csv('Patient_Visits.csv')
df_diagnoses = pd.read_csv('Diagnoses.csv')
df_resource = pd.read_csv('Resource_Usage.csv')

In [ ]:
# Checking the overview of patient visits dataset
df_Patient.head()

In [ ]:
# Checking the dtypes of columns
df_Patient.dtypes

In [6]:
# Converitng Date column dtype from object to datetime
df_Patient['Date'] = pd.to_datetime(df_Patient['Date'],errors='coerce')

In [ ]:
#Checking is there any missing or nan value
df_Patient.isna().sum()

In [ ]:
# Filling the missing value with median
df_Patient['LengthOfStay'].fillna(df_Patient['LengthOfStay'].median(),inplace=True)

In [16]:
# creating function for Detecting Outliers
def detect_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75) 
    IQR = Q3 - Q1 
    lower = Q1 -1.5 * IQR 
    higher = Q3 + 1.5 * IQR 
    return series[(series>higher) | (series<lower)]

In [17]:
# detected outliers assigned to variable
Outlier_Detected = detect_outliers(df_Patient['Cost'])

In [ ]:
# Checking how many outlier are there
print({'Outliers in Cost':len(Outlier_Detected)})

In [21]:
# Function to check the index of outliers
def outliers_index(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75) 
    IQR = Q3 - Q1 
    lower = Q1 -1.5 * IQR 
    higher = Q3 + 1.5 * IQR 
    return series[(series>higher) | (series<lower)].index

In [22]:
# Assigning index of outliers to a variable
Outlier_removed = outliers_index(df_Patient['Cost']) 

In [23]:
# Removing outliers and assign them to a new dataset which is cleaned
cleaned_patient_visits = df_Patient.drop(Outlier_removed)

In [ ]:
# Checking outliers are removed and there is no missing value
cleaned_patient_visits.head()

In [ ]:
# Checking is there any missing value in cleaned dataset
cleaned_patient_visits.isna().sum()

In [ ]:
# Checking the cleaned dataset dtypes
cleaned_patient_visits.dtypes

In [3]:
# reading the cleaned csv file for further visualization
df_cleaned_patient = pd.read_csv('cleaned_patient_visits.csv')

In [ ]:
# Checking the diagnosis dataset overview
df_diagnoses.head()

In [ ]:
# checking the dtypes
df_diagnoses.dtypes

In [ ]:
#  Checking is there any missing value 
df_diagnoses.isna().sum()

In [ ]:
# Filling the missing value with mode because it's categorical data
df_diagnoses['Severity'].fillna(df_diagnoses['Severity'].mode()[0],inplace=True)

In [ ]:
# Applying numerical value to saverity column because one visitid have multiple diagnoses code so we see in each visitid which have highest saverity
import pandas as pd  

severity_mapping = {'Low': 1 ,'Medium' : 2 , 'High' : 3}
df_diagnoses['Severity_Int'] = df_diagnoses['Severity'].map(severity_mapping) 

df_diagnoses['max_severity'] = df_diagnoses.groupby('VisitID')['Severity_Int'].transform('max')

df_diagnoses = df_diagnoses[['VisitID', 'DiagnosisCode', 'Severity','Severity_Int', 'max_severity']]

print(df_diagnoses)

In [ ]:
# Checking is numerical value applied 
df_diagnoses.head()

In [ ]:
# Checking the overview of resource dataset
df_resource.head()

In [62]:
# converting the uncleaned diagnosed dataset to cleaned dataset
df_diagnoses.to_csv('cleaned_diagnosis.csv',index=True)

In [4]:
# reading the cleaned datset for further analysis 
cleaned_diagnosis = pd.read_csv('cleaned_diagnosis.csv')

In [6]:
# importing sqlite3 and making connection for database
import sqlite3, csv 
con = sqlite3.connect('Hospital.db')
curr = con.cursor()

In [ ]:
# Install pretty table library for to show tables when executing sql query
!pip install ipython-sql pretty
import prettytable
prettytable.DEFAULT = 'DEFAULT'

In [8]:
#  loading the database
%load_ext sql

In [9]:
# Creating Connection to Database
%sql sqlite:///Hospital.db

In [ ]:
df_cleaned_patient.to_sql('patient_data',con,if_exists='replace',index=False)

In [ ]:
cleaned_diagnosis.to_sql('diagnosis_data',con,if_exists='replace',index=False)

In [ ]:
df_resource.to_sql('resource_data',con,if_exists='replace',index=False)

In [ ]:
# Identifying the Average cost per department using sql
avg_cost_per_dept = %sql select *, round(avg(Cost),2) as avg_cost_per_dept from patient_data group by Department

In [ ]:
# Checking the result
avg_cost_per_dept

In [ ]:
# Barchart visualization for Average cost per department

plt.figure(figsize=(10,6))
sns.barplot(data=avg_cost_per_dept.DataFrame(),x='Department',y='avg_cost_per_dept',palette="viridis")

for bar in plt.gca().patches:
    plt.annotate(format(bar.get_height()), 
                (bar.get_x() + bar.get_width()/2, bar.get_height() ),
                 ha= 'center',va= 'bottom'
                )

plt.title('Average Cost Per Department')
plt.xlabel('Departmet')
plt.ylabel('Cost')
plt.tight_layout()

In [ ]:
# Finding the 3 most unit-usage per resourse-type 
%%sql  resource_usage_per_resourcetype  <<
   -- Step 1: Merge the tables (equivalent to pd.merge)
WITH merged AS (
    SELECT 
        p.VisitID,
        p.PatientID,
        strftime('%Y-%m', p.Date) AS Month,
        p.Department,
        p.LengthOfStay,
        p.Cost,
        r.ResourceType,
        r.UnitsUsed
    FROM patient_data p
    JOIN resource_data r ON p.VisitID = r.VisitID
),
-- Step 2: Aggregate by Month and ResourceType
monthly_usage AS (
    SELECT 
        Month,
        ResourceType,
        SUM(UnitsUsed) AS Total_Units_Used
    FROM merged
    GROUP BY Month, ResourceType
),
-- Step 3: Identify top 3 most-used resource types overall
top_resources AS (
    SELECT ResourceType
    FROM monthly_usage
    GROUP BY ResourceType
    HAVING SUM(Total_Units_Used) >= (
        SELECT SUM(Total_Units_Used)
        FROM monthly_usage m2
        GROUP BY m2.ResourceType
        ORDER BY SUM(m2.Total_Units_Used) DESC
        LIMIT 1 OFFSET 2
    )
    ORDER BY SUM(Total_Units_Used) DESC
    LIMIT 3
)
-- Step 4: Filter only top 3 for further use
SELECT 
    m.Month,
    m.ResourceType,
    m.Total_Units_Used
FROM monthly_usage m
JOIN top_resources tr ON m.ResourceType = tr.ResourceType
ORDER BY m.Month, m.Total_Units_Used DESC;

In [116]:
# Converting the variable to dataframe for visualization
df_resource_usage_per_resourcetype = resource_usage_per_resourcetype.DataFrame()

In [ ]:
# Checking the overview of the 3 most unit-usage per resourse-type 
df_resource_usage_per_resourcetype.head()

In [ ]:
# Lineplot for the 3 most unit-usage per resourse-type using python
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Merge for time-based aggregation
merged = pd.merge(df_cleaned_patient, df_resource, on='VisitID')
# Optional: Set to monthly if needed
merged['Month'] = merged['Date'].dt.to_period('M').astype(str)
# Aggregate by Month and ResourceType
monthly_usage = merged.groupby(['Month', 'ResourceType'])['UnitsUsed'].sum().reset_index()
# Identify top 3 most-used resource types overall
top_resources = (
    monthly_usage.groupby('ResourceType')['UnitsUsed'].sum()
    .sort_values(ascending=False).head(3).index
)
# Filter only top 3 for plotting
top_usage = monthly_usage[monthly_usage['ResourceType'].isin(top_resources)]
# Plot
plt.figure(figsize=(12, 6))
sns.lineplot(data=top_usage, x='Month', y='UnitsUsed', hue='ResourceType', marker='o')
plt.xticks(rotation=45)
plt.title('Monthly Usage Trend of Top 3 Resources')
plt.tight_layout()
plt.show()

In [ ]:
# Merge all 3 datasets 
merged_all = df_cleaned_patient.merge(cleaned_diagnosis, on='VisitID').merge(df_resource, on='VisitID')
# Aggregate: UnitsUsed by DiagnosisCode and ResourceType
pivot_df = (
    merged_all.groupby(['DiagnosisCode', 'ResourceType'])['UnitsUsed']
    .sum().reset_index()
    .pivot(index='DiagnosisCode', columns='ResourceType', values='UnitsUsed')
    .fillna(0)
)
# Optional: Limit to most frequent 20 diagnosis codes
top_20_diag = merged_all['DiagnosisCode'].value_counts().head(20).index
pivot_df = pivot_df.loc[pivot_df.index.isin(top_20_diag)]
# Plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(pivot_df, annot=True, fmt=".0f", cmap="YlGnBu", linewidths=.5)
plt.title("Resource Usage by Diagnosis Code")
plt.xlabel("Resource Type")
plt.ylabel("Diagnosis Code")
plt.tight_layout()
plt.show()

In [ ]:
# Visualization for Cost Distribution by Department

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



# Remove extreme outliers (optional but recommended for clarity)
df_cleaned_patient = df_cleaned_patient[df_cleaned_patient['Cost'] < df_cleaned_patient['Cost'].quantile(0.99)]

# Plot boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_cleaned_patient, x='Department', y='Cost', palette='Set2')
plt.title("Cost Distribution by Department")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
